# **Diner Dash Challenge**

---

## Objective:

Using Reinforcement Learning(RL) algorithms, maximise the average rewards from 100 games/episodes of Diner Dash.

## Game information:

1. 40 states

2. 57 actions

For more information, visit the [diner dash repo](https://github.com/AdaCompNUS/diner-dash-simulator).

## Things to note:

1. If an "Error: A module (diner_dash) was specified for the environment but was not found, make sure the package is installed with `pip install` before calling `gym.make()`" error is raised, please restart the kernel and rerun all

2. Please ensure a strong internet connection throughout this challenge to avoid disconnecting from the collab GPUs

3. Do not idle your computer as collab automatically disconnects GPUs if the idle time is too long

4. GPUs run on CUDA 10.1

For other FAQs, refer to this [link](https://research.google.com/colaboratory/faq.html).

## Workshop Repo
If you have any questions regarding the use of this template, please raise an issue on the [workshop repo]().

---

# Details of Submission [Please Edit]

### Names of Group Members:
John, Mary, Bryan

### Names of Algorithms Used:
Random Agent, PPO


# Mounting Google Drive

To store trained models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Create Project Directory

In [ ]:
!mkdir drive/My\ Drive/DinerDashChallenge

import os
os.chdir("drive/My Drive/DinerDashChallenge")

mkdir: cannot create directory ‘drive/My Drive/DinerDashChallenge’: File exists


# Installing Dependencies

Downloading relevant project dependencies

## Dependencies for [diner dash simulator](https://github.com/AdaCompNUS/diner-dash-simulator)

In [ ]:
!git clone https://github.com/AdaCompNUS/diner-dash-simulator.git

fatal: destination path 'diner-dash-simulator' already exists and is not an empty directory.


In [ ]:
!pip install -e diner-dash-simulator/DinerDashEnv

Obtaining file:///content/drive/My%20Drive/DinerDashChallenge/diner-dash-simulator/DinerDashEnv
  Found existing installation: diner-dash 0.0.1
    Can't uninstall 'diner-dash'. No files were found to uninstall.
  Running setup.py develop for diner-dash


In [ ]:
!pip install more_itertools

## Dependencies for Policy

In [ ]:
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


# Helper Functions

For easier debugging

In [ ]:
def getAction(actionID):
    actionIDtoName = {
        0 : "None",
        1 : "Move to Table 1",
        2 : "Move to Table 2",
        3 : "Move to Table 3",
        4 : "Move to Table 4",
        5 : "Move to Table 5",
        6 : "Move to Table 6",
        7 : "Move to Counter",
        8 : "Pick Food for Table 1",
        9 : "Pick Food for Table 2",
        10 : "Pick Food for Table 3",
        11 : "Pick Food for Table 4",
        12 : "Pick Food for Table 5",
        13 : "Pick Food for Table 6",
        14 : "Move to Food Collection",
        15 : "Pick Table 1 for Group 1",
        16 : "Pick Table 2 for Group 1",
        17 : "Pick Table 3 for Group 1",
        18 : "Pick Table 4 for Group 1",
        19 : "Pick Table 5 for Group 1",
        20 : "Pick Table 6 for Group 1",
        21 : "Pick Table 1 for Group 2",
        22 : "Pick Table 2 for Group 2",
        23 : "Pick Table 3 for Group 2",
        24 : "Pick Table 4 for Group 2",
        25 : "Pick Table 5 for Group 2",
        26 : "Pick Table 6 for Group 2",
        27 : "Pick Table 1 for Group 3",
        28 : "Pick Table 2 for Group 3",
        29 : "Pick Table 3 for Group 3",
        30 : "Pick Table 4 for Group 3",
        31 : "Pick Table 5 for Group 3",
        32 : "Pick Table 6 for Group 3",
        33 : "Pick Table 1 for Group 4",
        34 : "Pick Table 2 for Group 4",
        35 : "Pick Table 3 for Group 4",
        36 : "Pick Table 4 for Group 4",
        37 : "Pick Table 5 for Group 4",
        38 : "Pick Table 6 for Group 4",
        39 : "Pick Table 1 for Group 5",
        40 : "Pick Table 2 for Group 5",
        41 : "Pick Table 3 for Group 5",
        42 : "Pick Table 4 for Group 5",
        43 : "Pick Table 5 for Group 5",
        44 : "Pick Table 6 for Group 5",
        45 : "Pick Table 1 for Group 6",
        46 : "Pick Table 2 for Group 6",
        47 : "Pick Table 3 for Group 6",
        48 : "Pick Table 4 for Group 6",
        49 : "Pick Table 5 for Group 6",
        50 : "Pick Table 6 for Group 6",
        51 : "Pick Table 1 for Group 7",
        52 : "Pick Table 2 for Group 7",
        53 : "Pick Table 3 for Group 7",
        54 : "Pick Table 4 for Group 7",
        55 : "Pick Table 5 for Group 7",
        56 : "Pick Table 6 for Group 7",
    }
    return actionIDtoName[actionID]

# Policies

## Random Agent

In [ ]:
from random import randint

In [ ]:
# Randomly select an action from the action space
def testRA(env, state):
    # init variables
    done = False
    sumReward = 0
    actionList = []

    while not done:
        action = randint(0, 56)
        actionList.append(action)
        state, reward, done, _ = env.step(action)
        sumReward += reward

    return sumReward, actionList

## [PPO](https://github.com/nikhilbarhate99/PPO-PyTorch)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.distributions import Categorical
import gym

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, action_dim),
                nn.Softmax(dim=-1)
                )
        
        # critic
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, 1)
                )
        
    def forward(self):
        raise NotImplementedError
        
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device) 
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy
        
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()
    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())
                
            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())

### Training PPO

In [ ]:
def trainPPO():
    ############## Hyperparameters ##############
    env_name = "diner_dash:DinerDash-v0"
    # creating environment
    env = gym.make(env_name).unwrapped
    state_dim = env.observation_space.shape[0]
    action_dim = 57
    render = False
    solved_reward = 500         # stop training if avg_reward > solved_reward
    log_interval = 1000         # print avg reward in the interval
    max_episodes = int(1e8)     # max training episodes
    max_timesteps = int(1e8)    # max timesteps in one episode
    n_latent_var = 64           # number of variables in hidden layer
    update_timestep = 2000      # update policy every n timesteps
    lr = 0.002
    betas = (0.9, 0.999)
    gamma = 0.99                # discount factor
    K_epochs = 4                # update policy for K epochs
    eps_clip = 0.2              # clip parameter for PPO
    random_seed = None          # do NOT train with a random seed
    #############################################
    
    if random_seed:
        torch.manual_seed(random_seed)
        env.seed(random_seed)
    
    memory = Memory()
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
    print(f"Device used: {device}")
    print(f"Learning rate: {lr}, Betas: {betas}")
    
    # logging variables
    running_reward = 0
    avg_length = 0
    timestep = 0
    
    # training loop
    for i_episode in range(1, max_episodes+1):
        state = env.reset()
        state = np.array(state)
        
        for t in range(max_timesteps):
          timestep += 1

          # Running policy_old:
          action = ppo.policy_old.act(state, memory)
          state, reward, done, _ = env.step(action)
          state = np.array(state)
          
          # Saving reward and is_terminal:
          memory.rewards.append(reward)
          memory.is_terminals.append(done)
          
          # update if its time
          if timestep % update_timestep == 0:
              ppo.update(memory)
              memory.clear_memory()
              timestep = 0
          
          running_reward += reward
          if render:
              env.render()
          if done:
              break
                
        avg_length += t
        
        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            torch.save(ppo.policy.state_dict(), 'PPO_{}_{}.pth'.format(env_name, solved_reward))
            break
            
        # logging
        if i_episode % log_interval == 0:
            avg_length = int(avg_length/log_interval)
            running_reward = int((running_reward/log_interval))
            
            print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
            running_reward = 0
            avg_length = 0

In [14]:
trainPPO()

Device used: cuda:0
Learning rate: 0.002, Betas: (0.9, 0.999)
Device used: cuda:0
Learning rate: 0.002, Betas: (0.9, 0.999)
Episode 1000 	 avg length: 137 	 reward: -960
Episode 1000 	 avg length: 137 	 reward: -960
Episode 2000 	 avg length: 137 	 reward: -369
Episode 2000 	 avg length: 137 	 reward: -369
Episode 3000 	 avg length: 139 	 reward: 50
Episode 3000 	 avg length: 139 	 reward: 50
Episode 4000 	 avg length: 141 	 reward: 163
Episode 4000 	 avg length: 141 	 reward: 163
Episode 5000 	 avg length: 141 	 reward: 249
Episode 5000 	 avg length: 141 	 reward: 249
Episode 6000 	 avg length: 143 	 reward: 318
Episode 6000 	 avg length: 143 	 reward: 318
Episode 7000 	 avg length: 141 	 reward: 267
Episode 7000 	 avg length: 141 	 reward: 267
Episode 8000 	 avg length: 142 	 reward: 235
Episode 8000 	 avg length: 142 	 reward: 235
Episode 9000 	 avg length: 142 	 reward: 273
Episode 9000 	 avg length: 142 	 reward: 273
Episode 10000 	 avg length: 142 	 reward: 310
Episode 10000 	 av

### Testing PPO

In [26]:
def testPPO(env, state):
    ############## Hyperparameters ##############
    env_name = "diner_dash:DinerDash-v0"
    # creating environment
    # env = gym.make(env_name).unwrapped
    state_dim = 40
    action_dim = 57
    exp_reward = 500            # expected reward to load saved model     
    n_latent_var = 64           # number of variables in hidden layer
    lr = 0.0007
    betas = (0.9, 0.999)
    gamma = 0.99                # discount factor
    K_epochs = 4                # update policy for K epochs
    eps_clip = 0.2              # clip parameter for PPO
    #############################################
    
    filename = "PPO_{}_{}.pth".format(env_name, exp_reward)
    
    memory = Memory()
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
    
    ppo.policy_old.load_state_dict(torch.load(filename))

    ep_reward = 0
    state = np.array(state)
    done = False

    while not done:
      action = ppo.policy_old.act(state, memory)
      state, reward, done, _ = env.step(action)
      state = np.array(state)
      ep_reward += reward

    actionList = [action.item() for action in memory.actions]

    return ep_reward, actionList

# Testing of Policies and Verification of Submission

In [33]:
from random import randint
import json
import os

# Sample test
def test():
    # Initiliase environment
    env = gym.make('diner_dash:DinerDash-v0').unwrapped
    env.flash_sim = False

    ############################ CHANGEABLE AREA ##############################
    # Changeable parameters
    numEpisodes = 100                             # num of test episodes
    algos = [testRA, testPPO]                     # Add or remove algos (must have unique names)
    saveJson = True                               # Whether to save actions_dict
    group_member_list = ["john", "mary", "bryan"] # List of group members

    ### Replace the list of randomSeeds with that given for submission
    # e.g. randomSeeds = [1, 2, 3]
    randomSeeds = [randint(0, 1e8) for i in range(numEpisodes)]

    ############################################################################

    rewards_dict = {algo.__name__ : [] for algo in algos}
    actions_dict = {algo.__name__ : [] for algo in algos}

    # Test begins
    for e, seed in enumerate(randomSeeds):
        # Sets random seed
        env.seed(seed)
        
        # Resets the environment based on random seed
        state = env.reset()

        for algo in algos:
            # create copy of environment for testing
            t_env = env.env.duplicate()

            # Given an environment and initial state
            # Returns the sum of rewards for that episode and the actions list
            rewards, actions = algo(t_env, state)

            rewards_dict[algo.__name__].append(rewards)
            actions_dict[algo.__name__].append(actions)

    # Print average rewards from n episodes for each algo
    avgReward_dict = {algo : int(sum(rewards)/len(rewards)) for algo, rewards in rewards_dict.items()}
    print(f"Average Rewards for each algo: {avgReward_dict}")

    # Print an action dict containing actions list for each random seed env for each algo
    print(f"Actions list for each env for each algo: {actions_dict}")
    
    submission_dict = {
        "names": group_member_list,
        "actionDict": actions_dict}

    if saveJson:
      print("Saving Json file...")
      fileName = "submission.json"
      with open(fileName, "w") as write_file:
          json.dump(submission_dict, write_file)
          print(f"{fileName} was saved in {os.getcwd()}")
      
      print("-" * 100)
      
      print(f"Verifying {fileName}...")
      print(f"Group Members include: {submission_dict['names']}")
      print(f"Names of Algos used: {list(submission_dict['actionDict'].keys())}")
      for val in submission_dict['actionDict'].values():
        submissionEpisodes = len(val)
        if submissionEpisodes != len(randomSeeds):
          raise ValueError("Number of episodes in submission does not match the number of random seeds!")
      print(f"Number of episodes(random seeds): {submissionEpisodes}")
      print("Number of episodes in submission matches the number of random seeds")
      print("Verification Complete! Please double check the verification results")
    
    return None

In [34]:
test()

Average Rewards for each algo: {'testRA': -1269, 'testPPO': 521}
Actions list for each env for each algo: {'testRA': [[46, 13, 8, 29, 40, 16, 34, 21, 48, 36, 53, 27, 25, 11, 18, 17, 48, 29, 13, 36, 51, 6, 36, 24, 10, 5, 46, 28, 4, 53, 32, 17, 6, 14, 9, 50, 54, 4, 56, 37, 8, 22, 8, 1, 25, 17, 1, 20, 54, 1, 17, 43, 54, 54, 16, 28, 56, 11, 33, 46, 0, 14, 55, 37, 50, 26, 36, 11, 2, 8, 41, 37, 1, 21, 12, 28, 24, 18, 12, 6, 32, 3, 10, 38, 1, 35, 37, 11, 6, 28, 7, 1, 23, 1, 31, 27, 0, 10, 10, 7, 31, 21, 49, 47, 13, 36, 38, 32, 4, 0, 47, 38, 10, 41, 41, 20, 12, 23, 48, 14, 4, 5, 24, 24, 44, 10, 13, 49, 48, 34, 26, 55, 22, 49, 22, 46, 46, 33, 22, 38, 53, 6, 42, 42, 36, 39, 27, 52, 22, 28, 49, 34, 43, 35, 2, 24, 39, 51], [15, 52, 48, 41, 25, 46, 2, 20, 54, 29, 46, 5, 24, 1, 28, 35, 21, 21, 19, 48, 45, 33, 31, 34, 48, 0, 26, 0, 30, 47, 29, 40, 42, 20, 45, 17, 50, 7, 14, 4, 53, 18, 30, 23, 22, 39, 39, 30, 22, 52, 53, 35, 8, 33, 27, 48, 23, 7, 43, 8, 42, 2, 9, 42, 43, 40, 19, 24, 43, 15, 14, 48, 13